In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id,prompt_id,text,generated
0,e663b2f5,0,The ideal career choice is one that aligns pas...,0
1,dc1bd601,0,The ideal career choice is one that aligns wit...,0
2,3f28d007,0,The ideal career choice is one that aligns wit...,0
3,46232258,0,The ideal career choice is one that aligns wit...,0
4,f9fff5c6,0,The ideal career choice is one that aligns wit...,0


In [3]:
df_prompts = pd.read_csv('train_prompts.csv')
df_prompts.head()

,prompt_id,prompt_name,instructions,source_text
0,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
1,1,Bottled water should be banned or not,\nWrite a letter to the government in which yo...,#Pro and Con: Bottled Water Ban | Britannica\n...


In [4]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,prompt_id,text
0,63cf09af,0,The ideal career choice is one that aligns wit...
1,908e398d,0,#Pursuing Passion in Professions \n The ideal ...
2,5e281241,0,The ideal career choice is one that aligns wit...
3,5f57d123,0,#Embracing Diversity in Career Choices \n Dive...
4,5bef5435,0,The ideal career choice is one that aligns wit...


In [5]:
df_submission = pd.read_csv('submission.csv')
df_submission.head()

,generated
0,0
1,0
2,0
3,0
4,0


In [6]:
df_train.head(20)

,id,prompt_id,text,generated
0,e663b2f5,0,The ideal career choice is one that aligns pas...,0
1,dc1bd601,0,The ideal career choice is one that aligns wit...,0
2,3f28d007,0,The ideal career choice is one that aligns wit...,0
3,46232258,0,The ideal career choice is one that aligns wit...,0
4,f9fff5c6,0,The ideal career choice is one that aligns wit...,0
5,b0cb8f89,0,#Adaptability and Career Success \n In today's...,0
6,1396fe1a,0,The ideal career choice is one that aligns wit...,0
7,9e52fa50,0,The ideal career choice is one that ignites a ...,0
8,18bf0fda,0,#Impact-Oriented Careers \n An ideal career ch...,0
9,07905282,0,The ideal career choice is one that ignites pa...,0


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier  # Example model
import google.generativeai as genai
import os
import re
import string

In [19]:
os.environ['GOOGLE_API_KEY'] = ""
genai.configure(api_key = os.getenv('GOOGLE_API_KEY'))

In [20]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-pro
models/gemini-pro-vision


In [119]:
train_data = pd.read_csv('train.csv')
train_prompts = pd.read_csv('train_prompts.csv')
train_data = train_data.merge(train_prompts, on="prompt_id")

In [120]:
test_data = pd.read_csv('test.csv')
test_data = test_data.merge(train_prompts, on="prompt_id")

In [121]:
train_data.head()

,id,prompt_id,text,generated,prompt_name,instructions,source_text
0,e663b2f5,0,The ideal career choice is one that aligns pas...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
1,dc1bd601,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
2,3f28d007,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
3,46232258,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
4,f9fff5c6,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...


In [122]:
test_data.head()

,id,prompt_id,text,prompt_name,instructions,source_text
0,63cf09af,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
1,908e398d,0,#Pursuing Passion in Professions \n The ideal ...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
2,5e281241,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
3,5f57d123,0,#Embracing Diversity in Career Choices \n Dive...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...
4,5bef5435,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...


In [123]:
train_data['combined_text'] = train_data[['text', 'prompt_name', 'instructions', 'source_text']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [124]:
train_data.head()

,id,prompt_id,text,generated,prompt_name,instructions,source_text,combined_text
0,e663b2f5,0,The ideal career choice is one that aligns pas...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns pas...
1,dc1bd601,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns wit...
2,3f28d007,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns wit...
3,46232258,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns wit...
4,f9fff5c6,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns wit...


In [125]:
test_data['combined_text'] = test_data[['text', 'prompt_name', 'instructions', 'source_text']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [126]:
test_data.head()

,id,prompt_id,text,prompt_name,instructions,source_text,combined_text
0,63cf09af,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns wit...
1,908e398d,0,#Pursuing Passion in Professions \n The ideal ...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,#Pursuing Passion in Professions \n The ideal ...
2,5e281241,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns wit...
3,5f57d123,0,#Embracing Diversity in Career Choices \n Dive...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,#Embracing Diversity in Career Choices \n Dive...
4,5bef5435,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,The ideal career choice is one that aligns wit...


In [23]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    tokens = text.split()
    processed_text = " ".join(tokens)
    return processed_text

In [127]:
train_data["combined_text"] = train_data["combined_text"].apply(preprocess_text)

In [128]:
test_data["combined_text"] = test_data["combined_text"].apply(preprocess_text)

In [129]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
def extract_gemini_embeddings(text):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",task_type="classification")
    vector = embeddings.embed_query(text)# Adjust prompt and parameters as needed
    return vector

In [130]:
train_data["gemini_embeddings"] = train_data["combined_text"].apply(extract_gemini_embeddings)

In [135]:
test_data["gemini_embeddings"] = test_data["combined_text"].apply(extract_gemini_embeddings)

In [131]:
train_data.head()

,id,prompt_id,text,generated,prompt_name,instructions,source_text,combined_text,gemini_embeddings
0,e663b2f5,0,The ideal career choice is one that aligns pas...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns pas...,"[0.020818021, -0.071440294, -0.020338288, 0.00..."
1,dc1bd601,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns wit...,"[0.017830085, -0.069388986, -0.020331088, 0.00..."
2,3f28d007,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns wit...,"[0.020024931, -0.06938759, -0.021297045, 0.003..."
3,46232258,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns wit...,"[0.019194758, -0.0708801, -0.021809917, 0.0043..."
4,f9fff5c6,0,The ideal career choice is one that aligns wit...,0,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns wit...,"[0.018189473, -0.072291344, -0.018864194, 0.00..."


In [136]:
test_data.head()

,id,prompt_id,text,prompt_name,instructions,source_text,combined_text,gemini_embeddings
0,63cf09af,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns wit...,"[0.018550066, -0.06859474, -0.019664865, 0.005..."
1,908e398d,0,#Pursuing Passion in Professions \n The ideal ...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,pursuing passion in professions n the ideal ca...,"[0.017023593, -0.0675562, -0.017745707, 0.0158..."
2,5e281241,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns wit...,"[0.017483547, -0.06779616, -0.019913165, 0.005..."
3,5f57d123,0,#Embracing Diversity in Career Choices \n Dive...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,embracing diversity in career choices n divers...,"[0.021775626, -0.074008286, -0.024300033, 0.01..."
4,5bef5435,0,The ideal career choice is one that aligns wit...,The Ideal Career Choice,\nWrite an explanatory essay to inform fellow ...,#How to Choose a Career: 7 Ways to Narrow Your...,the ideal career choice is one that aligns wit...,"[0.017188795, -0.06881043, -0.020702654, 0.006..."


In [99]:
train_data.shape

(212, 9)

In [74]:
test_data.shape

(188, 4)

In [132]:
import numpy as np
train_data['gemini_embeddings'] = train_data['gemini_embeddings'].apply(lambda x: np.array(x))

X_train, X_test, y_train, y_test = train_test_split(
    np.vstack(train_data["gemini_embeddings"]),  # Convert lists to a suitable format
    train_data["generated"],
    test_size=0.2,
)

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [133]:
model.score(X_train,y_train)

1.0

In [69]:
model.score(X_test, y_test)

0.9534883720930233

In [104]:
model.score(X_test, y_test)

0.9767441860465116

In [134]:
model.score(X_test, y_test)

1.0

In [137]:
x_val = np.vstack(test_data["gemini_embeddings"])

In [138]:
y_pred = model.predict(x_val)

In [77]:
y_pred

array([0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [111]:
y_pred

array([0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [139]:
y_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [78]:
df_submission.head()

,generated
0,0
1,0
2,0
3,0
4,0


In [115]:
df_submission.head()

,generated
0,0
1,0
2,0
3,0
4,0


In [79]:
df_submission.shape

(188, 1)

In [112]:
y_pred.shape

(188,)

In [140]:
df_submission['generated'] = y_pred

In [116]:
df_submission1 = df_submission

In [141]:
df_submission2 = df_submission

In [118]:
df_submission1.to_csv('submission_1.csv')

In [142]:
df_submission2.to_csv('submission_2.csv')

In [ ]:
#base submission 0.50
#first submission 0.94